In [31]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [32]:
import time

import warnings
warnings.filterwarnings('ignore')

In [33]:
driver = webdriver.Chrome() 

In [34]:
url = "https://map.kakao.com/"
driver.get(url)

In [35]:
searchloc = '청주'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [36]:
rv_list = []  # 정보 저장

In [37]:
def rvNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    rv_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, room in enumerate(rv_lists):
        temp = []
        
        name = room.select('.head_item > .tit_name > .link_name')[0].text
        score = room.select('.rating > .score > em')[0].text
        addr = room.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER) #후기더보기
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[7]/div[3]/a').send_keys(Keys.ENTER)
        
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER)

        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 room_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        rv_list.append(temp)

In [38]:
#new
def extract_review():
    # 후기 더보기 버튼을 클릭하는 루프
    while True:
        try:
            # 후기 더보기 버튼의 XPATH 경로로 찾기//*[@id="mArticle"]/div[8]/div[3]/a
            more_button = driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[7]/div[3]/a')
            
            if '후기 접기' in more_button.text:
                break
            
            
            more_button.click()  # 클릭
            time.sleep(2)  # 페이지 로딩 대기
        except Exception as e:
            # 더보기 버튼이 없거나 클릭 실패 시 종료
            print("NEXT PLACE", e)
            break

    # 페이지 소스를 가져와서 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    rev = []
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev


In [39]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)


for i in range(1, 9):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        if i > 5:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        else:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        
        rvNamePrint()  #  정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        if page2 > 5:
            page2 = 1
            driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
NEXT PLACE Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mArticle"]/div[7]/div[3]/a"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DEF79412+29090]
	(No symbol) [0x00007FF6DEEEE239]
	(No symbol) [0x00007FF6DEDAB1DA]
	(No symbol) [0x00007FF6DEDFEFE7]
	(No symbol) [0x00007FF6DEDFF23C]
	(No symbol) [0x00007FF6DEE497C7]
	(No symbol) [0x00007FF6DEE2672F]
	(No symbol) [0x00007FF6DEE465A2]
	(No symbol) [0x00007FF6DEE26493]
	(No symbol) [0x00007FF6DEDF09D1]
	(No symbol) [0x00007FF6DEDF1B31]
	GetHandleVerifier [0x00007FF6DF29871D+3302573]
	GetHandleVerifier [0x00007FF6DF2E4243+3612627]
	GetHandleVerifier [0x00007FF6DF2DA417+3572135]
	GetHandleVerifier [0x00007FF6DF035EB6+801862]
	(No symbol) [0x00007FF6DEEF945F]
	(No symbol) [0x00007FF6DEEF4FB4]
	(No symbol) [0x000

In [40]:
rv_list

[['청남대',
  '4.1',
  '청주시 상당구 문의면 청남대길 646',
  ['무료주차/ 유료입장료수갑찬 대통령은 좀 뺐으면..한번쯤 가기 좋은 곳, 두시간 좀 넘게 보고 왔어요 가족끼리 가기 좋은곳, 본관 내부도 볼만함하지만 훌륭하지 않은 부끄러운 대통령도 계속 등장(?)해서 자녀 교육 할때 애좀 먹었네요.. 수갑찬 대통령은 좀 뺐으면..',
   '가을 단풍 경치와 주위 경관은 너무 너무 좋습니다. 그러나 청남대 가는데 입구 진입에서 한시간 주차하다시피 느리게 이동해야 합니다. 입장해서는 가장 아름다운 곳 중심에 요새같은 별장에서 지냈을 역대 대통령 중 몇몇의 모습이 연상되었어요. 그 시대에 그 시설, 그들을 위해 군인들의 보초 섰을 상황이 현실적으로 그려지더군요. 역사 사진도 있고요. 영상 자료인지 뭐도 보러가고 하던데 전 그냥 패스 했습니다. 씁쓸한 마음에 얼굴이 좀 구겨지던 찰나 앞에 가시던 어른분이 하시던 말씀이 기억납니다. "아주 호사를 누렸구만!" 네 동감합니다. 풍경과 단풍 사진 명소에서 사진 찍으시고요 역사의 현실과 진실도 알고 가시면 좋을 것 같네요. 전 자주 갈 것 같진 않아요.',
   '과거에도 3회 정도 다녀왔는데, 이번에 갔더니 시설물들이 더욱 늘었고 볼거리도 많아졌네요!계절마다 즐길거리가 풍성합니다.꼭 가보시길 강추합니다~!',
   '들어가는  입구가  압권임내부  돌아보는덴   5시간 정도 걸링경치 좋고 관리 잘 됨',
   '여기 너무 좋네요.내부에 식당 및 식음료 판매시설이 전무한게 많이 아쉽긴 합니다.볼거리가 너무 많아 도시락 지참 후 중간에 식사하시고 계속 보셔야합니다.',
   '풍경 좋고 공기도 좋아요. 가로수길이 넘 예쁘네요.입장료는 6천원이고, 주차비는 없어졌나봐요.',
   '수유실도 있고 구경하기 좋아요',
   '월요일 휴무라 나와서 가다가 돌아올뻔함. 아직 꽃은 이르지만 벌써 덥기시작. 입장료가 쎄다.. 경로우대도 3000원 낸다. 욕을 많이 먹나보다 직원에게 잘 대해달라고 안내판이 있다.',
  

In [41]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_records(rv_list)
df.to_excel('test4.xlsx')

In [ ]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER)
    count = 0
    rev = []
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev